<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/11_2_%ED%8A%B8%EB%A6%AC_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
'''
부트스트랩 샘플
- 랜덤하게 뽑은 샘플을 다시 집어놔서 뽑는 거 -> 중복을 허락

랜덤포레스트 분류는 전체 특성 개수의 제곱근만큼 특성을 선택
랜덤포레스트 회귀는 전체 특성을 선택
'''

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

rf.fit(train_input, train_target)
print(rf.feature_importances_)

'''
oob
- 부트스트랩 샘플에서 제외된 샘플을 검증세트 역할로 사용
'''

rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)  # 교차검증 점수와 비슷하게 나옴

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
0.8934000384837406


In [5]:
'''
엑스트라 트리
- 부트스트랩 샘플을 사용하지 않음
- 전체 훈련 세트를 사용
- 무작위 분할
- 빠름
'''
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


In [7]:
'''
그래디언트 부스팅
- 깊이가 얕은 결정트리를 사용해 이전 트리의 오차를 보완
- 기본적으로 깊이가 3인 트리를 100개 사용
- 과대적합에 강하고 일반화 성능을 기대할 수 있음
- rf보다 약간 높은 성능
- 트리를 순차적으로 계산하기 때문에 느림
- 병렬처리하지 않기 때문에 n_jobs 매개변수가 없음
'''
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state =42)   # n_estimators = 500 -> 트리개수 500개
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(train_input, train_target)
print(gb.feature_importances_)

0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


In [11]:
'''
히스토그램 기반 그레디언트 부스팅
- 특성을 256개 구간으로 나눔
 -> 노드를 분할할 때 최적 분할을 매우 빠르게 찾을 수 있음
- 256개 구간 중에서 하나를 떼어 놓고 누락된 값을 위해 사용
 -> 누락된 특성이 있더라도 이를 따로 전처리할 필요 없음

아직테스트 과정
 '''
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

hgb.score(test_input, test_target)

0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923

설명을 보면 데이콘 1에 아주 적합할 것으로 예상됨

In [12]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686
0.935828414851749 0.8801251203079884
